In [2]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]

In [4]:
text.shape

AttributeError: 'list' object has no attribute 'shape'

In [8]:

# text feature and labels
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# x_train_text = np.load(path+'x_train_text.npy')
# vad = np.load(path+'y_egemaps.npy')
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')

data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)
# standardization
scaled_feature = False

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

file_loc = '../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
del(word_vectors)



scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

Found 3437 unique tokens
../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
G Null word embeddings: 1


In [9]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [10]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
____________________

In [11]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_W2V8000_no_dropout.csv', header=['time','v','a','d', 'ave'])


Instructions for updating:
Use tf.cast instead.
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 52s 8ms/step - loss: 1.8407 - v_loss: 0.5958 - a_loss: 0.6228 - d_loss: 0.6221 - v_ccc: 0.4042 - a_ccc: 0.3772 - d_ccc: 0.3779 - val_loss: 2.0118 - val_v_loss: 0.6996 - val_a_loss: 0.6470 - val_d_loss: 0.6652 - val_v_ccc: 0.3004 - val_a_ccc: 0.3530 - val_d_ccc: 0.3348
Epoch 2/50
6400/6400 [==============================] - 49s 8ms/step - loss: 1.4192 - v_loss: 0.3877 - a_loss: 0.5161 - d_loss: 0.5153 - v_ccc: 0.6123 - a_ccc: 0.4839 - d_ccc: 0.4847 - val_loss: 1.9454 - val_v_loss: 0.6523 - val_a_loss: 0.6380 - val_d_loss: 0.6550 - val_v_ccc: 0.3477 - val_a_ccc: 0.3620 - val_d_ccc: 0.3450
Epoch 3/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.2645 - v_loss: 0.3337 - a_loss: 0.4702 - d_loss: 0.4606 - v_ccc: 0.6663 - a_ccc: 0.5298 - d_ccc: 0.5394 - val_loss: 1.8929 - val_v_loss: 0.6246 - val_a_loss: 0.6124 - val_d_lo

Epoch 12/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.6097 - v_loss: 0.1496 - a_loss: 0.2301 - d_loss: 0.2300 - v_ccc: 0.8504 - a_ccc: 0.7699 - d_ccc: 0.7700 - val_loss: 1.9634 - val_v_loss: 0.5892 - val_a_loss: 0.6758 - val_d_loss: 0.6983 - val_v_ccc: 0.4108 - val_a_ccc: 0.3242 - val_d_ccc: 0.3017
Epoch 13/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.5678 - v_loss: 0.1402 - a_loss: 0.2161 - d_loss: 0.2116 - v_ccc: 0.8598 - a_ccc: 0.7839 - d_ccc: 0.7884 - val_loss: 1.9956 - val_v_loss: 0.5946 - val_a_loss: 0.7051 - val_d_loss: 0.6960 - val_v_ccc: 0.4054 - val_a_ccc: 0.2949 - val_d_ccc: 0.3040
Epoch 14/50
2039/2039 [==============================] - 4s 2ms/step
[0.39784789085388184, 0.38718047738075256, 0.41487571597099304]
1
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 53s 8ms/step - loss: 1.8243 - v_loss: 0.5851 - a_loss: 0.6113 - d_loss: 0.6279 - v_ccc: 0.4149 - a_ccc: 0.388

Epoch 15/50
2039/2039 [==============================] - 5s 2ms/step
[0.3961293399333954, 0.3973367512226105, 0.40765446424484253]
4
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 59s 9ms/step - loss: 1.8819 - v_loss: 0.5961 - a_loss: 0.6404 - d_loss: 0.6454 - v_ccc: 0.4039 - a_ccc: 0.3596 - d_ccc: 0.3546 - val_loss: 2.0857 - val_v_loss: 0.7042 - val_a_loss: 0.6948 - val_d_loss: 0.6867 - val_v_ccc: 0.2958 - val_a_ccc: 0.3052 - val_d_ccc: 0.3133
Epoch 2/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.4403 - v_loss: 0.3995 - a_loss: 0.5243 - d_loss: 0.5165 - v_ccc: 0.6005 - a_ccc: 0.4757 - d_ccc: 0.4835 - val_loss: 1.9499 - val_v_loss: 0.6753 - val_a_loss: 0.6138 - val_d_loss: 0.6608 - val_v_ccc: 0.3247 - val_a_ccc: 0.3862 - val_d_ccc: 0.3392
Epoch 3/50
6400/6400 [==============================] - 54s 8ms/step - loss: 1.2711 - v_loss: 0.3315 - a_loss: 0.4728 - d_loss: 0.4667 - v_ccc: 0.6685 - a_ccc: 0.5272 - 

6400/6400 [==============================] - 51s 8ms/step - loss: 1.1425 - v_loss: 0.2909 - a_loss: 0.4288 - d_loss: 0.4228 - v_ccc: 0.7091 - a_ccc: 0.5712 - d_ccc: 0.5772 - val_loss: 1.9150 - val_v_loss: 0.6224 - val_a_loss: 0.6328 - val_d_loss: 0.6597 - val_v_ccc: 0.3776 - val_a_ccc: 0.3672 - val_d_ccc: 0.3403
Epoch 5/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.0486 - v_loss: 0.2633 - a_loss: 0.3951 - d_loss: 0.3902 - v_ccc: 0.7367 - a_ccc: 0.6049 - d_ccc: 0.6098 - val_loss: 1.9204 - val_v_loss: 0.6049 - val_a_loss: 0.6265 - val_d_loss: 0.6890 - val_v_ccc: 0.3951 - val_a_ccc: 0.3735 - val_d_ccc: 0.3110
Epoch 6/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.9625 - v_loss: 0.2392 - a_loss: 0.3573 - d_loss: 0.3660 - v_ccc: 0.7608 - a_ccc: 0.6427 - d_ccc: 0.6340 - val_loss: 1.9306 - val_v_loss: 0.6002 - val_a_loss: 0.6515 - val_d_loss: 0.6789 - val_v_ccc: 0.3998 - val_a_ccc: 0.3485 - val_d_ccc: 0.3211
Epoch 7/50
6400/6400 [==============

Epoch 11/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.6468 - v_loss: 0.1599 - a_loss: 0.2451 - d_loss: 0.2418 - v_ccc: 0.8401 - a_ccc: 0.7549 - d_ccc: 0.7582 - val_loss: 1.9773 - val_v_loss: 0.6186 - val_a_loss: 0.6701 - val_d_loss: 0.6887 - val_v_ccc: 0.3814 - val_a_ccc: 0.3299 - val_d_ccc: 0.3113
Epoch 12/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.6012 - v_loss: 0.1489 - a_loss: 0.2281 - d_loss: 0.2242 - v_ccc: 0.8511 - a_ccc: 0.7719 - d_ccc: 0.7758 - val_loss: 1.9483 - val_v_loss: 0.5951 - val_a_loss: 0.6656 - val_d_loss: 0.6877 - val_v_ccc: 0.4049 - val_a_ccc: 0.3344 - val_d_ccc: 0.3123
Epoch 13/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.5688 - v_loss: 0.1391 - a_loss: 0.2177 - d_loss: 0.2120 - v_ccc: 0.8609 - a_ccc: 0.7823 - d_ccc: 0.7880 - val_loss: 1.9182 - val_v_loss: 0.5908 - val_a_loss: 0.6605 - val_d_loss: 0.6669 - val_v_ccc: 0.4092 - val_a_ccc: 0.3395 - val_d_ccc: 0.3331
Epoch 14/50
6400/6400 

Epoch 13/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.5667 - v_loss: 0.1434 - a_loss: 0.2146 - d_loss: 0.2088 - v_ccc: 0.8566 - a_ccc: 0.7854 - d_ccc: 0.7912 - val_loss: 1.9793 - val_v_loss: 0.5867 - val_a_loss: 0.6866 - val_d_loss: 0.7060 - val_v_ccc: 0.4133 - val_a_ccc: 0.3134 - val_d_ccc: 0.2940
Epoch 14/50
2039/2039 [==============================] - 4s 2ms/step
[0.3850194811820984, 0.3838784396648407, 0.3946744203567505]
12
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 53s 8ms/step - loss: 1.8184 - v_loss: 0.5876 - a_loss: 0.6221 - d_loss: 0.6087 - v_ccc: 0.4124 - a_ccc: 0.3779 - d_ccc: 0.3913 - val_loss: 2.0602 - val_v_loss: 0.6867 - val_a_loss: 0.6623 - val_d_loss: 0.7112 - val_v_ccc: 0.3133 - val_a_ccc: 0.3377 - val_d_ccc: 0.2888
Epoch 2/50
6400/6400 [==============================] - 48s 8ms/step - loss: 1.4205 - v_loss: 0.3904 - a_loss: 0.5177 - d_loss: 0.5124 - v_ccc: 0.6096 - a_ccc: 0.4823 -

Epoch 13/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.5664 - v_loss: 0.1368 - a_loss: 0.2168 - d_loss: 0.2127 - v_ccc: 0.8632 - a_ccc: 0.7832 - d_ccc: 0.7873 - val_loss: 1.9608 - val_v_loss: 0.6072 - val_a_loss: 0.6726 - val_d_loss: 0.6810 - val_v_ccc: 0.3928 - val_a_ccc: 0.3274 - val_d_ccc: 0.3190
Epoch 14/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.5306 - v_loss: 0.1299 - a_loss: 0.2030 - d_loss: 0.1976 - v_ccc: 0.8701 - a_ccc: 0.7970 - d_ccc: 0.8024 - val_loss: 1.9473 - val_v_loss: 0.6107 - val_a_loss: 0.6603 - val_d_loss: 0.6763 - val_v_ccc: 0.3893 - val_a_ccc: 0.3397 - val_d_ccc: 0.3237
Epoch 15/50
6400/6400 [==============================] - 49s 8ms/step - loss: 0.5022 - v_loss: 0.1247 - a_loss: 0.1906 - d_loss: 0.1869 - v_ccc: 0.8753 - a_ccc: 0.8094 - d_ccc: 0.8131 - val_loss: 1.9469 - val_v_loss: 0.5855 - val_a_loss: 0.6730 - val_d_loss: 0.6885 - val_v_ccc: 0.4145 - val_a_ccc: 0.3270 - val_d_ccc: 0.3115
Epoch 16/50
2039/2039 

Epoch 15/50
6400/6400 [==============================] - 54s 9ms/step - loss: 0.5131 - v_loss: 0.1254 - a_loss: 0.2004 - d_loss: 0.1873 - v_ccc: 0.8746 - a_ccc: 0.7996 - d_ccc: 0.8127 - val_loss: 1.9382 - val_v_loss: 0.5863 - val_a_loss: 0.6562 - val_d_loss: 0.6957 - val_v_ccc: 0.4137 - val_a_ccc: 0.3438 - val_d_ccc: 0.3043
Epoch 16/50
2039/2039 [==============================] - 4s 2ms/step
[0.3910483121871948, 0.3862316608428955, 0.388954222202301]
18
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.8249 - v_loss: 0.5760 - a_loss: 0.6233 - d_loss: 0.6255 - v_ccc: 0.4240 - a_ccc: 0.3767 - d_ccc: 0.3745 - val_loss: 2.0368 - val_v_loss: 0.6885 - val_a_loss: 0.6755 - val_d_loss: 0.6728 - val_v_ccc: 0.3115 - val_a_ccc: 0.3245 - val_d_ccc: 0.3272
Epoch 2/50
6400/6400 [==============================] - 49s 8ms/step - loss: 1.4221 - v_loss: 0.3921 - a_loss: 0.5168 - d_loss: 0.5132 - v_ccc: 0.6079 - a_ccc: 0.4832 - 